In [ ]:
# ==========================================
# 1. GOOGLE COLAB: EĞİTİM VE PAKETLEME
# ==========================================

import os
# GPU'yu sadece eğitim için kullan, ama modeli CPU uyumlu yap
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Gerekli Kütüphaneler
try:
    import kagglehub
except ImportError:
    os.system('pip install kagglehub')
    import kagglehub

import glob
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, Concatenate, Embedding, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler

print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.19.0


In [ ]:
# --- AYARLAR ---
SEQ_LEN = 5
FILES_CONFIG = {
    'capture-fishery-production.csv': {'type_id': 0, 'col_keyword': 'Capture', 'topic': 'avcılık üretimi'},
    'aquaculture-farmed-fish-production.csv': {'type_id': 1, 'col_keyword': 'Aquaculture', 'topic': 'yetiştiricilik'},
    'fish-and-seafood-consumption-per-capita.csv': {'type_id': 2, 'col_keyword': 'Food supply', 'topic': 'tüketim'},
    'fish-stocks-within-sustainable-levels.csv': {'type_id': 3, 'col_keyword': 'sustainable levels', 'topic': 'stok sürdürülebilirliği'}
}

In [ ]:
# --- VERİ İNDİRME VE İŞLEME ---
print("Veri seti indiriliyor...")
path = kagglehub.dataset_download("sergegeukjian/fish-and-overfishing")
all_files = glob.glob(os.path.join(path, "**", "*.csv"), recursive=True)

X_values, X_types, Y_texts = [], [], []

for fname, config in FILES_CONFIG.items():
    found = next((f for f in all_files if fname in f), None)
    if not found: continue

    df = pd.read_csv(found)
    try: val_col = [c for c in df.columns if config['col_keyword'] in c][0]
    except: continue

    df = df.rename(columns={'Entity': 'Entity', 'Year': 'Year', val_col: 'Value'})
    df = df.dropna(subset=['Value']).sort_values(['Entity', 'Year'])
    df['Value'] = pd.to_numeric(df['Value'], errors='coerce')

    for country in df['Entity'].unique():
        vals = df[df['Entity'] == country]['Value'].values
        if len(vals) < SEQ_LEN + 1: continue

        for i in range(len(vals) - SEQ_LEN):
            window = vals[i : i+SEQ_LEN]
            v_start, v_end = window[0] + 1e-5, window[-1]
            pct = ((v_end - v_start) / v_start) * 100

            trend = "artış" if pct > 10 else "düşüş" if pct < -10 else "stabil"
            if config['type_id'] == 3: # Stoklar
                 trend = "kritik" if v_end < 60 else "riskli" if pct < -5 else "sürdürülebilir"

            text = f"start {config['topic']} verileri {trend} seyretti end"
            X_values.append(window); X_types.append(config['type_id']); Y_texts.append(text)

Veri seti indiriliyor...


100%|██████████| 516k/516k [00:00<00:00, 662kB/s]

Extracting files...


In [ ]:
# --- PREPROCESSING ---
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(np.array(X_values).T).T.reshape(-1, SEQ_LEN, 1)

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(Y_texts)
Y_pad = pad_sequences(tokenizer.texts_to_sequences(Y_texts), padding='post')
max_len = Y_pad.shape[1]

In [ ]:
# --- MODEL (GPU + TFLITE DOSTU) ---
input_seq = Input(shape=(SEQ_LEN, 1), name='ts_input')
# unroll=True sayesinde GPU kullanırız ama TFLite hata vermez
lstm_out = LSTM(64, return_state=False, unroll=True)(input_seq)
input_type = Input(shape=(1,), name='type_input')
type_vec = Flatten()(Embedding(4, 16)(input_type))
concat = Concatenate()([lstm_out, type_vec])
decoder = LSTM(64, return_sequences=True, unroll=True)(RepeatVector(max_len)(Dense(64, activation='relu')(concat)))
output = Dense(len(tokenizer.word_index) + 1, activation='softmax')(decoder)

model = Model([input_seq, input_type], output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit([X_scaled, np.array(X_types)], Y_pad, epochs=15, batch_size=64, verbose=1)

Epoch 1/15
526/526 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - accuracy: 0.6634 - loss: 1.1206
Epoch 2/15
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9714 - loss: 0.0785
Epoch 3/15
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9727 - loss: 0.0680
Epoch 4/15
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9719 - loss: 0.0703
Epoch 5/15
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9738 - loss: 0.0618
Epoch 6/15
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9739 - loss: 0.0610
Epoch 7/15
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9740 - loss: 0.0610
Epoch 8/15
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9739 - loss: 0.0604
Epoch 9/15
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9735 - loss: 0.0598
Epoch 10/15
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9746 - loss: 0.0587
Epoch 11/15
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9745 - loss: 0.0590
Epoch 12/15
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/st

In [ ]:
# --- DIŞA AKTARMA ---
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('fishery_model.tflite', 'wb') as f: f.write(tflite_model)
with open('model_meta.json', 'w') as f:
    json.dump({'max_len': max_len, 'tokenizer_json': tokenizer.to_json(), 'topic_map': {v['topic']: v['type_id'] for k, v in FILES_CONFIG.items()}}, f)

print("\nBAŞARILI! 'fishery_model.tflite' ve 'model_meta.json' dosyalarını indir.")

Saved artifact at '/tmp/tmpsbbbqj1k'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 5, 1), dtype=tf.float32, name='ts_input'), TensorSpec(shape=(None, 1), dtype=tf.float32, name='type_input')]
Output Type:
  TensorSpec(shape=(None, 7, 16), dtype=tf.float32, name=None)
Captures:
  134093692416848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134093692417424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134093692418960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134093692417232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134093692419344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134093692419920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134093667599568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134093667599184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134093667600144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134093692419728: Te